In [ ]:
import React, { useState } from 'react';
import { Send, Key, CheckCircle, XCircle, Zap } from 'lucide-react';

export default function PhysicsChatbot() {
  const [apiKey, setApiKey] = useState('');
  const [isVerified, setIsVerified] = useState(false);
  const [messages, setMessages] = useState([]);
  const [input, setInput] = useState('');
  const [loading, setLoading] = useState(false);
  const [verifying, setVerifying] = useState(false);

  const verifyApiKey = async () => {
    if (!apiKey.trim()) {
      alert('Please enter an API key');
      return;
    }

    setVerifying(true);
    try {
      const response = await fetch('https://api.groq.com/openai/v1/models', {
        headers: {
          'Authorization': `Bearer ${apiKey}`,
          'Content-Type': 'application/json'
        }
      });

      if (response.ok) {
        setIsVerified(true);
        setMessages([{
          role: 'assistant',
          content: 'API key verified! I\'m your Physics chatbot. Ask me anything about physics - mechanics, thermodynamics, electromagnetism, quantum physics, relativity, and more!'
        }]);
      } else {
        alert('Invalid API key. Please check and try again.');
        setIsVerified(false);
      }
    } catch (error) {
      alert('Error verifying API key. Please try again.');
      setIsVerified(false);
    } finally {
      setVerifying(false);
    }
  };

  const isPhysicsQuestion = async (question) => {
    const physicsKeywords = [
      'physics', 'force', 'motion', 'energy', 'mass', 'velocity', 'acceleration',
      'momentum', 'gravity', 'friction', 'newton', 'quantum', 'atom', 'particle',
      'wave', 'light', 'electric', 'magnetic', 'thermodynamic', 'heat', 'temperature',
      'pressure', 'relativity', 'einstein', 'mechanics', 'kinematics', 'dynamics',
      'optics', 'sound', 'frequency', 'wavelength', 'radiation', 'nuclear', 'photon',
      'electron', 'proton', 'neutron', 'circuit', 'voltage', 'current', 'resistance',
      'capacitor', 'inductor', 'torque', 'angular', 'oscillation', 'pendulum',
      'spring', 'collision', 'conservation', 'joule', 'watt', 'power', 'work'
    ];

    const lowerQuestion = question.toLowerCase();
    return physicsKeywords.some(keyword => lowerQuestion.includes(keyword));
  };

  const sendMessage = async () => {
    if (!input.trim() || loading) return;

    const userMessage = { role: 'user', content: input };
    setMessages(prev => [...prev, userMessage]);
    setInput('');
    setLoading(true);

    try {
      const isPhysics = await isPhysicsQuestion(input);

      if (!isPhysics) {
        setMessages(prev => [...prev, {
          role: 'assistant',
          content: 'I apologize, but I can only answer questions related to Physics. Please ask me about topics like mechanics, thermodynamics, electromagnetism, quantum physics, optics, or any other physics-related subject.'
        }]);
        setLoading(false);
        return;
      }

      const response = await fetch('https://api.groq.com/openai/v1/chat/completions', {
        method: 'POST',
        headers: {
          'Authorization': `Bearer ${apiKey}`,
          'Content-Type': 'application/json'
        },
        body: JSON.stringify({
          model: 'llama-3.3-70b-versatile',
          messages: [
            {
              role: 'system',
              content: 'You are a specialized Physics tutor and expert. You only answer questions related to physics topics including but not limited to: classical mechanics, thermodynamics, electromagnetism, optics, quantum mechanics, relativity, nuclear physics, and astrophysics. Provide clear, accurate, and educational explanations. If asked about non-physics topics, politely decline and remind the user you only discuss physics.'
            },
            ...messages.filter(m => m.role !== 'assistant' || !m.content.includes('API key verified')),
            userMessage
          ],
          temperature: 0.7,
          max_tokens: 1024
        })
      });

      if (!response.ok) {
        throw new Error('Failed to get response from Groq API');
      }

      const data = await response.json();
      const assistantMessage = {
        role: 'assistant',
        content: data.choices[0].message.content
      };

      setMessages(prev => [...prev, assistantMessage]);
    } catch (error) {
      setMessages(prev => [...prev, {
        role: 'assistant',
        content: 'Sorry, there was an error processing your request. Please try again.'
      }]);
    } finally {
      setLoading(false);
    }
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-blue-50 via-indigo-50 to-purple-50 p-4">
      <div className="max-w-4xl mx-auto">
        <div className="bg-white rounded-2xl shadow-2xl overflow-hidden">
          {/* Header */}
          <div className="bg-gradient-to-r from-blue-600 to-indigo-600 p-6 text-white">
            <div className="flex items-center gap-3">
              <Zap className="w-8 h-8" />
              <div>
                <h1 className="text-3xl font-bold">Physics Chatbot</h1>
                <p className="text-blue-100 text-sm mt-1">Powered by Groq AI</p>
              </div>
            </div>
          </div>

          {/* API Key Section */}
          {!isVerified && (
            <div className="p-6 bg-blue-50 border-b border-blue-100">
              <label className="block text-sm font-semibold text-gray-700 mb-2">
                Enter Groq API Key
              </label>
              <div className="flex gap-2">
                <div className="flex-1 relative">
                  <Key className="absolute left-3 top-1/2 transform -translate-y-1/2 text-gray-400 w-5 h-5" />
                  <input
                    type="password"
                    value={apiKey}
                    onChange={(e) => setApiKey(e.target.value)}
                    placeholder="gsk_..."
                    className="w-full pl-10 pr-4 py-3 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
                    onKeyPress={(e) => e.key === 'Enter' && verifyApiKey()}
                  />
                </div>
                <button
                  onClick={verifyApiKey}
                  disabled={verifying}
                  className="px-6 py-3 bg-blue-600 text-white rounded-lg font-semibold hover:bg-blue-700 disabled:bg-gray-400 disabled:cursor-not-allowed transition-colors flex items-center gap-2"
                >
                  {verifying ? (
                    <>
                      <div className="w-5 h-5 border-2 border-white border-t-transparent rounded-full animate-spin" />
                      Verifying...
                    </>
                  ) : (
                    <>
                      <CheckCircle className="w-5 h-5" />
                      Verify
                    </>
                  )}
                </button>
              </div>
              <p className="text-xs text-gray-600 mt-2">
                Get your API key from{' '}
                <a href="https://console.groq.com" target="_blank" rel="noopener noreferrer" className="text-blue-600 hover:underline">
                  console.groq.com
                </a>
              </p>
            </div>
          )}

          {isVerified && (
            <div className="p-4 bg-green-50 border-b border-green-100 flex items-center gap-2">
              <CheckCircle className="w-5 h-5 text-green-600" />
              <span className="text-sm font-medium text-green-800">API Key Verified</span>
            </div>
          )}

          {/* Chat Messages */}
          <div className="h-96 overflow-y-auto p-6 space-y-4 bg-gray-50">
            {messages.length === 0 && !isVerified && (
              <div className="text-center text-gray-500 mt-20">
                <Zap className="w-16 h-16 mx-auto mb-4 text-gray-300" />
                <p className="text-lg font-medium">Enter your Groq API key to start</p>
              </div>
            )}

            {messages.map((msg, idx) => (
              <div
                key={idx}
                className={`flex ${msg.role === 'user' ? 'justify-end' : 'justify-start'}`}
              >
                <div
                  className={`max-w-[80%] rounded-2xl px-4 py-3 ${
                    msg.role === 'user'
                      ? 'bg-blue-600 text-white'
                      : 'bg-white text-gray-800 shadow-md border border-gray-200'
                  }`}
                >
                  <p className="whitespace-pre-wrap">{msg.content}</p>
                </div>
              </div>
            ))}

            {loading && (
              <div className="flex justify-start">
                <div className="bg-white rounded-2xl px-4 py-3 shadow-md border border-gray-200">
                  <div className="flex gap-2">
                    <div className="w-2 h-2 bg-blue-600 rounded-full animate-bounce" style={{animationDelay: '0ms'}} />
                    <div className="w-2 h-2 bg-blue-600 rounded-full animate-bounce" style={{animationDelay: '150ms'}} />
                    <div className="w-2 h-2 bg-blue-600 rounded-full animate-bounce" style={{animationDelay: '300ms'}} />
                  </div>
                </div>
              </div>
            )}
          </div>

          {/* Input Section */}
          {isVerified && (
            <div className="p-4 bg-white border-t border-gray-200">
              <div className="flex gap-2">
                <input
                  type="text"
                  value={input}
                  onChange={(e) => setInput(e.target.value)}
                  onKeyPress={(e) => e.key === 'Enter' && sendMessage()}
                  placeholder="Ask me about physics... (e.g., What is Newton's second law?)"
                  className="flex-1 px-4 py-3 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-transparent"
                  disabled={loading}
                />
                <button
                  onClick={sendMessage}
                  disabled={loading || !input.trim()}
                  className="px-6 py-3 bg-blue-600 text-white rounded-lg font-semibold hover:bg-blue-700 disabled:bg-gray-400 disabled:cursor-not-allowed transition-colors flex items-center gap-2"
                >
                  <Send className="w-5 h-5" />
                  Send
                </button>
              </div>
            </div>
          )}
        </div>

        {/* Info Footer */}
        <div className="mt-4 text-center text-sm text-gray-600">
          <p>💡 This chatbot only responds to physics-related questions</p>
        </div>
      </div>
    </div>
  );
}

SyntaxError: invalid decimal literal (ipython-input-2958728301.py, line 205)

In [1]:
import gradio as gr
import requests
import json

# Global variable to store API key
api_key_store = {"key": None, "verified": False}

def verify_api_key(api_key):
    """Verify the Groq API key"""
    if not api_key or not api_key.strip():
        return "❌ Please enter an API key", False

    try:
        response = requests.get(
            "https://api.groq.com/openai/v1/models",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            timeout=10
        )

        if response.status_code == 200:
            api_key_store["key"] = api_key
            api_key_store["verified"] = True
            return "✅ API Key Verified Successfully! You can now ask physics questions.", True
        else:
            api_key_store["verified"] = False
            return f"❌ Invalid API key. Status code: {response.status_code}", False

    except Exception as e:
        api_key_store["verified"] = False
        return f"❌ Error verifying API key: {str(e)}", False

def is_physics_question(question):
    """Check if the question is related to physics"""
    physics_keywords = [
        'physics', 'force', 'motion', 'energy', 'mass', 'velocity', 'acceleration',
        'momentum', 'gravity', 'friction', 'newton', 'quantum', 'atom', 'particle',
        'wave', 'light', 'electric', 'magnetic', 'thermodynamic', 'heat', 'temperature',
        'pressure', 'relativity', 'einstein', 'mechanics', 'kinematics', 'dynamics',
        'optics', 'sound', 'frequency', 'wavelength', 'radiation', 'nuclear', 'photon',
        'electron', 'proton', 'neutron', 'circuit', 'voltage', 'current', 'resistance',
        'capacitor', 'inductor', 'torque', 'angular', 'oscillation', 'pendulum',
        'spring', 'collision', 'conservation', 'joule', 'watt', 'power', 'work',
        'inertia', 'density', 'buoyancy', 'refraction', 'diffraction', 'interference'
    ]

    question_lower = question.lower()
    return any(keyword in question_lower for keyword in physics_keywords)

def chat_with_bot(message, history):
    """Main chat function"""
    if not api_key_store["verified"]:
        return history + [[message, "⚠️ Please verify your API key first using the button above."]]

    if not message or not message.strip():
        return history

    # Check if question is physics-related
    if not is_physics_question(message):
        response = ("🚫 I apologize, but I can only answer questions related to Physics. "
                   "Please ask me about topics like mechanics, thermodynamics, electromagnetism, "
                   "quantum physics, optics, relativity, or any other physics-related subject.")
        return history + [[message, response]]

    try:
        # Prepare messages for API
        messages = [
            {
                "role": "system",
                "content": ("You are a specialized Physics tutor and expert. You only answer questions "
                           "related to physics topics including but not limited to: classical mechanics, "
                           "thermodynamics, electromagnetism, optics, quantum mechanics, relativity, "
                           "nuclear physics, and astrophysics. Provide clear, accurate, and educational "
                           "explanations. If asked about non-physics topics, politely decline and remind "
                           "the user you only discuss physics.")
            }
        ]

        # Add conversation history
        for human, assistant in history:
            messages.append({"role": "user", "content": human})
            messages.append({"role": "assistant", "content": assistant})

        # Add current message
        messages.append({"role": "user", "content": message})

        # Call Groq API
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key_store['key']}",
                "Content-Type": "application/json"
            },
            json={
                "model": "llama-3.3-70b-versatile",
                "messages": messages,
                "temperature": 0.7,
                "max_tokens": 1024
            },
            timeout=30
        )

        if response.status_code == 200:
            data = response.json()
            bot_response = data["choices"][0]["message"]["content"]
            return history + [[message, bot_response]]
        else:
            error_msg = f"❌ Error from API (Status {response.status_code}): {response.text}"
            return history + [[message, error_msg]]

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        return history + [[message, error_msg]]

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft(), title="Physics Chatbot") as demo:
    gr.Markdown(
        """
        # ⚡ Physics Chatbot
        ### Powered by Groq AI

        This chatbot **only responds to physics-related questions**. Ask about mechanics,
        thermodynamics, electromagnetism, quantum physics, optics, and more!
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            api_key_input = gr.Textbox(
                label="🔑 Groq API Key",
                placeholder="Enter your Groq API key (gsk_...)",
                type="password"
            )
        with gr.Column(scale=1):
            verify_btn = gr.Button("Verify API Key", variant="primary", size="lg")

    verification_status = gr.Textbox(
        label="Status",
        interactive=False,
        show_label=False
    )

    gr.Markdown("---")

    chatbot = gr.Chatbot(
        label="Chat",
        height=400,
        show_label=False,
        avatar_images=(None, "🔬")
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Message",
            placeholder="Ask me about physics... (e.g., What is Newton's second law?)",
            show_label=False,
            scale=9
        )
        send_btn = gr.Button("Send", variant="primary", scale=1)

    gr.Markdown(
        """
        ---
        💡 **Tip**: This chatbot uses keyword detection to ensure questions are physics-related.

        📚 **Example questions**:
        - What is Newton's second law?
        - Explain quantum entanglement
        - How does a capacitor work?
        - What is the speed of light?

        🔗 Get your API key from [console.groq.com](https://console.groq.com)
        """
    )

    # Event handlers
    verify_btn.click(
        fn=verify_api_key,
        inputs=[api_key_input],
        outputs=[verification_status]
    )

    msg.submit(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    send_btn.click(
        fn=chat_with_bot,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch()

/tmp/ipython-input-3441535777.py:118: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Physics Chatbot") as demo:
/tmp/ipython-input-3441535777.py:147: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3441535777.py:147: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d9c9e2ece4fe84cca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
